In [26]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error,accuracy_score,r2_score

from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

In [40]:
train = pd.read_csv("train_data.csv")

In [41]:
test = pd.read_csv("test_data_hidden.csv")
test.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [42]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,2,2015-06-30,5735,568,1,1,0,0
1,2,2,2015-06-30,9863,877,1,1,0,0
2,3,2,2015-06-30,13261,1072,1,1,0,1
3,4,2,2015-06-30,13106,1488,1,1,0,0
4,5,2,2015-06-30,6635,645,1,1,0,0


In [43]:
train['Date']= pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

In [44]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982644 entries, 0 to 982643
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Store          982644 non-null  int64         
 1   DayOfWeek      982644 non-null  int64         
 2   Date           982644 non-null  datetime64[ns]
 3   Sales          982644 non-null  int64         
 4   Customers      982644 non-null  int64         
 5   Open           982644 non-null  int64         
 6   Promo          982644 non-null  int64         
 7   StateHoliday   982644 non-null  object        
 8   SchoolHoliday  982644 non-null  int64         
dtypes: datetime64[ns](1), int64(7), object(1)
memory usage: 67.5+ MB


In [45]:
# train.sort_values(by='Store', inplace=True)
# train = train[:88155]
# train.sort_index(ignore_index=True,inplace=True)

In [46]:
train

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,2,2015-06-30,5735,568,1,1,0,0
1,2,2,2015-06-30,9863,877,1,1,0,0
2,3,2,2015-06-30,13261,1072,1,1,0,1
3,4,2,2015-06-30,13106,1488,1,1,0,0
4,5,2,2015-06-30,6635,645,1,1,0,0
...,...,...,...,...,...,...,...,...,...
982639,1111,2,2013-01-01,0,0,0,0,a,1
982640,1112,2,2013-01-01,0,0,0,0,a,1
982641,1113,2,2013-01-01,0,0,0,0,a,1
982642,1114,2,2013-01-01,0,0,0,0,a,1


In [47]:
# test.sort_values(by='Store', inplace=True)
# test = train[:4000]
# test.sort_index(ignore_index=True,inplace=True)

In [48]:
test

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
34560,1111,3,2015-07-01,3701,351,1,1,0,1
34561,1112,3,2015-07-01,10620,716,1,1,0,1
34562,1113,3,2015-07-01,8222,770,1,1,0,0
34563,1114,3,2015-07-01,27071,3788,1,1,0,0


In [49]:
concat = pd.concat([train, test])

In [50]:
concat.shape

(1017209, 9)

In [51]:
concat.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,2,2015-06-30,5735,568,1,1,0,0
1,2,2,2015-06-30,9863,877,1,1,0,0
2,3,2,2015-06-30,13261,1072,1,1,0,1
3,4,2,2015-06-30,13106,1488,1,1,0,0
4,5,2,2015-06-30,6635,645,1,1,0,0


In [52]:
concat["StateHoliday"] = concat["StateHoliday"].replace({"0":0, "a":1, "b":2, "c":3})

concat["Day"]= pd.to_datetime(concat["Date"]).dt.day
concat["Month"]= pd.to_datetime(concat["Date"]).dt.month
concat["Year"]= pd.to_datetime(concat["Date"]).dt.year

concat.drop(['Date', 'Customers'], axis=1, inplace=True)

In [53]:
concat.head()

,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday,Day,Month,Year
0,1,2,5735,1,1,0,0,30,6,2015
1,2,2,9863,1,1,0,0,30,6,2015
2,3,2,13261,1,1,0,1,30,6,2015
3,4,2,13106,1,1,0,0,30,6,2015
4,5,2,6635,1,1,0,0,30,6,2015


In [54]:
dummies = pd.get_dummies(data=concat, columns=['DayOfWeek','Open','Promo','StateHoliday','SchoolHoliday','Year','Month','Day'], drop_first=True)

In [55]:
dummies.head()

,Store,Sales,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7,Open_1,Promo_1,...,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31
0,1,5735,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1,0
1,2,9863,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1,0
2,3,13261,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1,0
3,4,13106,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1,0
4,5,6635,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1,0


In [56]:
train.shape

(982644, 9)

In [58]:
# train = dummies[:88155]
# test = dummies[88155:]
train = dummies[:982644]
test = dummies[982644:]

In [59]:
train.shape

(982644, 57)

In [60]:
test.shape

(34565, 57)

In [61]:
X_train = train.drop('Sales', axis=1)
y_train = train['Sales']

X_test = test.drop('Sales', axis=1)
y_test = test['Sales']

In [62]:
X_test.head()

,Store,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7,Open_1,Promo_1,StateHoliday_1,...,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31
0,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
1,2,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
2,3,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
3,4,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
4,5,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1


In [63]:
y_test.head()

0     5263
1     6064
2     8314
3    13995
4     4822
Name: Sales, dtype: int64

## Linear Regression Model

In [64]:
lr = LinearRegression()
lr.fit(X_train, y_train)

# train_pred = lr.predict(X_train)
# print('R-squared error for training set is: ', r2_score(train_pred, y_train))
# print('RMSE for training set is: ', mean_squared_error( train_pred, y_train, squared=False))
# print('MAE for training set is: ',mean_absolute_error(train_pred, y_train))

test_pred = lr.predict(X_test)
print('R-squared error for testing set is: ', r2_score(test_pred, y_test))
print('RMSE for testing set is: ', mean_squared_error( test_pred, y_test, squared=False))
print('MAE for testing set is: ',mean_absolute_error(test_pred, y_test))


# R-squared error for testing set is:  0.14708710706523198
# RMSE for testing set is:  2519.953267141104
# MAE for testing set is:  1731.4698261558028

R-squared error for testing set is:  0.14705838351041112
RMSE for testing set is:  2520.0716734482558
MAE for testing set is:  1731.704737951252


In [65]:
lr.predict([[1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]])

array([9259.44771892])

## Regularization

In [66]:
ridge = Ridge()

ridge.fit(X_train, y_train)
test_pred = ridge.predict(X_test)
print('R-squared error for testing set is: ', r2_score(test_pred, y_test))
print('RMSE for testing set is: ', mean_squared_error( test_pred, y_test, squared=False))
print('MAE for testing set is: ',mean_absolute_error(test_pred, y_test))


R-squared error for testing set is:  0.1470324884657942
RMSE for testing set is:  2520.071498764995
MAE for testing set is:  1731.696462571646


## Applying Tree Based Regressor

### Descision Tree Regressor

In [67]:
X_train

,Store,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7,Open_1,Promo_1,StateHoliday_1,...,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31
0,1,1,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
1,2,1,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
2,3,1,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
3,4,1,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
4,5,1,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982639,1111,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
982640,1112,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
982641,1113,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
982642,1114,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [73]:
dt_reg = DecisionTreeRegressor()

dt_reg.fit(X_train, y_train)
train_pred = dt_reg.predict(X_train)
print('R-squared error for training set is: ', r2_score(train_pred, y_train))
print('RMSE for training set is: ', mean_squared_error(train_pred, y_train, squared=False))
print('MAE for training set is: ',mean_absolute_error(train_pred, y_train))

test_pred = dt_reg.predict(X_test)
print('R-squared error for testing set is: ', r2_score(test_pred, y_test))
print('RMSE for testing set is: ', mean_squared_error( test_pred, y_test, squared=False))
print('MAE for testing set is: ',mean_absolute_error(test_pred, y_test))


R-squared error for training set is:  1.0
RMSE for training set is:  0.0
MAE for training set is:  0.0
R-squared error for testing set is:  0.7141951419184349
RMSE for testing set is:  2019.4820450778993
MAE for testing set is:  1081.1908867351367


### Random Forest

In [147]:
rf_reg = RandomForestRegressor(n_estimators=100,
    criterion='mse',
    max_depth=6,
    )

rf_reg.fit(X_train, y_train)
test_pred = rf_reg.predict(X_test)

train_pred = rf_reg.predict(X_train)
print('R-squared error for training set is: ', r2_score(train_pred, y_train))
print('RMSE for training set is: ', mean_squared_error( train_pred, y_train, squared=False))
print('MAE for training set is: ',mean_absolute_error(train_pred, y_train),'\n'*2)

print('R-squared error for testing set is: ', r2_score(test_pred, y_test))
print('RMSE for testing set is: ', mean_squared_error( test_pred, y_test, squared=False))
print('MAE for testing set is: ',mean_absolute_error(test_pred, y_test))

R-squared error for training set is:  0.5117109466113656
RMSE for training set is:  1985.8622359455992
MAE for training set is:  1382.4869026159995 


R-squared error for testing set is:  0.42944803828449596
RMSE for testing set is:  2105.771951080073
MAE for testing set is:  1417.3404683320948


In [150]:
from sklearn.model_selection import RandomizedSearchCV

In [148]:
n_estimators = [10, 20, 30, 50, 100]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [10, 20, 30, 50, 100], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [154]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, 
                               cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [10, 20, 30, 50, 100]},
                   random_state=42, verbose=2)

In [155]:
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [10, 20, 30, 50, 100]},
                   random_state=42, verbose=2)

In [156]:
rf_random.best_estimator_

RandomForestRegressor(bootstrap=False, max_depth=50, max_features='sqrt',
                      min_samples_leaf=4, n_estimators=50)

In [157]:
rf_random.best_params_

{'n_estimators': 50,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 50,
 'bootstrap': False}

In [159]:
test_preds = rf_random.predict(X_test)

print('R-squared error for testing set is: ', r2_score(test_pred, y_test))
print('RMSE for testing set is: ', mean_squared_error( test_pred, y_test, squared=False))
print('MAE for testing set is: ',mean_absolute_error(test_pred, y_test))

R-squared error for testing set is:  0.42944803828449596
RMSE for testing set is:  2105.771951080073
MAE for testing set is:  1417.3404683320948


### ADA Boost

In [101]:
ada = AdaBoostRegressor()
ada.fit(X_train, y_train)
test_pred = ada.predict(X_test)
print('R-squared error for testing set is: ', r2_score(test_pred, y_test))
print('RMSE for testing set is: ', mean_squared_error( test_pred, y_test, squared=False))
print('MAE for testing set is: ',mean_absolute_error(test_pred, y_test))


R-squared error for testing set is:  0.2502526104798247
RMSE for testing set is:  3191.4753758553525
MAE for testing set is:  2456.467328346888


### Gradient Boosting

In [69]:
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train)

test_pred = gbr.predict(X_test)
print('R-squared error for testing set is: ', r2_score(test_pred, y_test))
print('RMSE for testing set is: ', mean_squared_error( test_pred, y_test, squared=False))
print('MAE for testing set is: ',mean_absolute_error(test_pred, y_test))


R-squared error for testing set is:  0.4763170786376838
RMSE for testing set is:  2061.645951451762
MAE for testing set is:  1429.1661992215718


### Support Vector

In [71]:
svr = SVR()
svr.fit(X_train, y_train)

test_pred = svr.predict(X_test)
print('R-squared error for testing set is: ', r2_score(test_pred, y_test))
print('RMSE for testing set is: ', mean_squared_error( test_pred, y_test, squared=False))
print('MAE for testing set is: ',mean_absolute_error(test_pred, y_test))


R-squared error for testing set is:  -3.31907431124922
RMSE for testing set is:  2794.4553591636704
MAE for testing set is:  2141.113323034652
